In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

/usr/local/Cellar/python/3.7.2_2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/usr/local/Cellar/python/3.7.2_2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/local/Cellar/python/3.7.2_2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
W0731 19:07:28.495802 4529210816 deprecation.py:323] From <ipython-input-1-03052c1d1b02>:6: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./mnist/data/train-images-idx3-ubyte.gz


W0731 19:07:55.993831 4529210816 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0731 19:07:55.996186 4529210816 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.


Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz


W0731 19:07:58.884862 4529210816 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [2]:
# 하이퍼파라미터들 설정
total_epoch = 100
batch_size = 100
learning_rate = 0.0002
n_hidden = 256
n_input = 28 * 28

# 생성자의 입력값으로 사용할 노이즈의 크기이다.
n_noise = 128

In [4]:
# 플레이스홀더 설정

# 실제 이미지를 넣을 플레이스홀더 
X = tf.placeholder(tf.float32, [None, n_input])

# 가짜 이미지를 생성할 플레이스홀더
Z = tf.placeholder(tf.float32, [None, n_noise])

# GAN도 비지도 학습이므로 오토인코더 처럼 Y를 사용하지 않는다.

In [6]:
# 생성자 신경망에 사용할 변수들 설정

# 첫 번째 가중치와 편향은 은닉층으로 출력하기 위한 변수
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))

# 두 번째 가중치와 편향은 출력층에 사용할 변수
# n_input : 실제 이미지의 크기이므로 28 * 28
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

In [8]:
# 구분자 신경망에 사용할 변수들을 설정

# 은닉층은 생성자와 동일하게 구성
D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))

# 구분자는 진짜와 얼마나 가까운지를 판단하는 값으로, 0~1 사이의 값 출력
D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1]))

In [9]:
# 생성자 신경망
# 무작위로 생성한 노이즈를 받아 가중치와 편향을 반영하여 은닉층 생성
# 은닉층에서 실제 이미지와 같은 크기의 결괏값을 출력한다.
def generator(noise_z):
    hidden = tf.nn.relu(tf.matmul(noise_z, G_W1) + G_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, G_W2) + G_b2)
    return output

In [11]:
# 구분자 신경망
# 0~1 사이의 스칼라값 하나를 출력하도록 하고,
# 이를 위한 활성화 함수로 sigmoid 함수를 사용
def discriminator(inputs):
    hidden = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, D_W2) + D_b2)
    return output

In [12]:
# 무작위한 노이즈를 만들어주는 함수
def get_noise(batch_size, n_noise):
    return np.random.normal(size=(batch_size, n_noise))

In [13]:
# 노이즈 Z를 이용해 가짜 이미지를 만들 생성자 G를 만들고
# 이 G가 만든 가짜 이미지와 진짜 이미지 X를 각각 구분자에 넣어 입력한 이미지가
# 진짜인지를 판별
G = generator(Z)
D_gene = discriminator(G)
D_real = discriminator(X)

In [15]:
# 손실값 구하기

# 1. 생성자가 만든 이미지를 구분자가 가짜라고 판단하도록 하는 손실값
# 2. 진짜라고 판단하도록 하는 손실값
# 진짜 이미지이면 D_real은 1에 가깝고, 가짜 이미지면 0에 가깝다.
loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1 - D_gene))
loss_G = tf.reduce_mean(tf.log(D_gene))

In [16]:
# 학습 준비
# loss_D를 구할 때는 구분자 신경망에 사용되는 변수들만 사용
# loss_G를 구할 때는 생성자 신경망에 사용되는 변수들만 사용
D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]

In [23]:
# loss를 최대화해야 하는데, 최적화에 쓸 수 있는 함수는 minimize뿐이므로 
# 최적화하려는 loss_D와 loss_G에 음수 부호를 붙여준다.
train_D = tf.train.AdamOptimizer(learning_rate).minimize(
    -loss_D, var_list=D_var_list)

train_G = tf.train.AdamOptimizer(learning_rate).minimize(
    -loss_G, var_list=G_var_list)

In [25]:
# 학습
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 미니배치를 위함
total_batch = int(mnist.train.num_examples / batch_size)
# 결과값을 받을 변수
loss_var_D, loss_val_G = 0, 0

In [31]:
for epoch in range(total_epoch):
    for i in range(total_batch):
        # 미니배치로 학습을 반복하도록 한다.
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)
        
        # 구분자는 X 값을, 생성자는 노이즈인 Z 값을 받으므로 노이즈를 생성해주는
        # get_noise 함수를 통해 배치 크기만큼 노이즈를 만들고 이를 입력해준다.
        _, loss_val_D = sess.run([train_D, loss_D],
                                feed_dict={X: batch_xs, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G],
                                feed_dict={Z: noise})
        
    print('Epoch: ', '%04d' %epoch,
         'D loss: {:.4}'.format(loss_val_D),
         'G loss: {:.4}'.format(loss_val_G))
    
    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        # 노이즈를 만들고, 이것을 생성자 G에 넣은 결괏값을 만든다.
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict={Z: noise})
        
        # 결괏값들을 28 * 28 크기의 가짜 이미지로 만들어 폴더에 저장
        fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1))
        
        for i in range(sample_size):
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i], (28, 28)))
            
        plt.savefig('./samples/{}.png'.format(str(epoch).zfill(3)),
                   bbox_inches='tight')
        plt.close(fig)

print('최적화 완료')

Epoch:  0000 D loss: -0.1698 G loss: -2.933
Epoch:  0001 D loss: -0.3859 G loss: -3.239
Epoch:  0002 D loss: -0.1962 G loss: -2.992
Epoch:  0003 D loss: -0.3327 G loss: -2.527
Epoch:  0004 D loss: -0.156 G loss: -2.915
Epoch:  0005 D loss: -0.3474 G loss: -2.411
Epoch:  0006 D loss: -0.4196 G loss: -2.448
Epoch:  0007 D loss: -0.4119 G loss: -2.452
Epoch:  0008 D loss: -0.2719 G loss: -2.453
Epoch:  0009 D loss: -0.4592 G loss: -2.326
Epoch:  0010 D loss: -0.3755 G loss: -2.527
Epoch:  0011 D loss: -0.4504 G loss: -2.365
Epoch:  0012 D loss: -0.4698 G loss: -2.283
Epoch:  0013 D loss: -0.3046 G loss: -2.452
Epoch:  0014 D loss: -0.4217 G loss: -2.836
Epoch:  0015 D loss: -0.4194 G loss: -2.692
Epoch:  0016 D loss: -0.4752 G loss: -2.393
Epoch:  0017 D loss: -0.368 G loss: -2.242
Epoch:  0018 D loss: -0.2937 G loss: -2.754
Epoch:  0019 D loss: -0.3189 G loss: -2.889
Epoch:  0020 D loss: -0.3571 G loss: -3.113
Epoch:  0021 D loss: -0.4143 G loss: -2.483
Epoch:  0022 D loss: -0.6027 G los